In [ ]:
pip install transformers

In [2]:
import numpy as np
import pandas as pd
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import transformers as ppb

In [4]:
df = pd.DataFrame(columns = ['sentence','class'])
df

,sentence,class


In [5]:
with open('/content/drive/MyDrive/data_chatbot.txt', 'r') as f:
    lst = []
    num=0
    for line in f:
        if line=='\n':
            lst = list(set(lst))
            df1 = pd.DataFrame({'sentence':lst,'class':[num]*len(lst)})
            df = pd.concat([df, df1]).reset_index(drop = True)
            lst=[]
            num+=1
        else:
            line = line.lower()
            lst.append(line[:-1])
df1 = pd.DataFrame({'sentence':lst,'class':[num]*len(lst)})
df = pd.concat([df, df1]).reset_index(drop = True)
df = df.sample(frac=1).reset_index(drop=True)
df

,sentence,class
0,что можешь?,1
1,можно ли взять кредит?,3
2,какие машины я могу купить?,4
3,какие машины?,4
4,расскажи о кредитах,3
...,...,...
257,какие модели доступны?,4
258,что ты можешь сделать?,1
259,каков ваш вектор направленности?,2
260,каков характер деятельности компании?,2


In [6]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'DeepPavlov/rubert-base-cased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [7]:
tokenized = df['sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized

0                         [101, 1997, 102264, 166, 102]
1            [101, 7021, 11089, 19475, 15123, 166, 102]
2      [101, 19201, 14798, 877, 31424, 26835, 166, 102]
3                         [101, 19201, 14798, 166, 102]
4               [101, 61557, 852, 612, 36672, 884, 102]
                             ...                       
257                [101, 19201, 14406, 27603, 166, 102]
258          [101, 1997, 4609, 102264, 13555, 166, 102]
259    [101, 2739, 1388, 28007, 32303, 47833, 166, 102]
260      [101, 2739, 1388, 8618, 10252, 6597, 166, 102]
261    [101, 2739, 19475, 15123, 1469, 21754, 166, 102]
Name: sentence, Length: 262, dtype: object

In [8]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(262, 16)

In [9]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(262, 16)

In [10]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [11]:
features = last_hidden_states[0][:,0,:].numpy()
labels = df['class']
labels=labels.astype('int')

In [32]:
x_train, x_test, y_train, y_test = train_test_split(features, labels,test_size=0.2)
lr_clf = LogisticRegression(C=1,multi_class='ovr',max_iter=300,random_state=41)
lr_clf.fit(x_train, y_train)
lr_clf.score(x_test, y_test)

1.0

In [36]:
from xgboost import XGBClassifier

params = {'n_estimators': 500,
 'reg_alpha': 0.05,
 'reg_lambda': 9,
 #'colsample_bytree': 0.3,
 'eval_metric':'merror',
 'subsample': 0.8,
 'learning_rate': 0.1,
 'max_depth': 5,
 'num_leaves': 20,
 'random_state': 42,
 'min_child_samples': 47,
 'min_data_per_groups': 96,
 'tree_method': "gpu_hist",
 "gpu_id": 0}

model_xgb = XGBClassifier(**params)
model_xgb.fit(x_train,y_train, eval_set=[(x_test,y_test)], verbose=True,early_stopping_rounds=10)

[0]	validation_0-merror:0.296296
Will train until validation_0-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.148148
[2]	validation_0-merror:0.148148
[3]	validation_0-merror:0.074074
[4]	validation_0-merror:0.037037
[5]	validation_0-merror:0.037037
[6]	validation_0-merror:0.037037
[7]	validation_0-merror:0.037037
[8]	validation_0-merror:0
[9]	validation_0-merror:0
[10]	validation_0-merror:0
[11]	validation_0-merror:0
[12]	validation_0-merror:0
[13]	validation_0-merror:0
[14]	validation_0-merror:0
[15]	validation_0-merror:0
[16]	validation_0-merror:0
[17]	validation_0-merror:0
[18]	validation_0-merror:0
Stopping. Best iteration:
[8]	validation_0-merror:0



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='merror',
              gamma=0, gpu_id=0, learning_rate=0.1, max_delta_step=0,
              max_depth=5, min_child_samples=47, min_child_weight=1,
              min_data_per_groups=96, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, num_leaves=20, objective='multi:softprob',
              random_state=42, reg_alpha=0.05, reg_lambda=9, scale_pos_weight=1,
              seed=None, silent=None, subsample=0.8, tree_method='gpu_hist',
              verbosity=1)

## Try bert-only classification

In [37]:
import torch
from transformers import BertModel, BertTokenizerFast
from transformers import Trainer, TrainingArguments, AdamW

In [38]:
class CustomBERTModel(torch.nn.Module):
  def __init__(self):
    super(CustomBERTModel, self).__init__()

    self.num_labels = 5
    self.weights = None
    self.bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')
    self.dropout = torch.nn.Dropout(0.1)
    self.out = torch.nn.Linear(768, self.num_labels)
    # self.softmax = torch.nn.Softmax(19)

  def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
    res = self.bert(
          input_ids, 
          attention_mask=attention_mask,
          token_type_ids=token_type_ids
    )

    sequence_output = self.dropout(res['pooler_output'])

    logits = self.out(sequence_output)

    outputs = (logits, )

    if labels is not None:
      loss_fct = torch.nn.CrossEntropyLoss(weight=self.weights)
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
      outputs = (loss,) + outputs

    return outputs

In [39]:
tokenizer = BertTokenizerFast.from_pretrained('DeepPavlov/rubert-base-cased')

In [45]:
x_train, x_test, y_train, y_test = train_test_split(df.sentence.tolist(), df['class'].tolist(),test_size=0.2)

train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_test, truncation=True, padding=True)

In [46]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, y_train)
val_dataset = Dataset(val_encodings, y_test)

In [47]:
model = CustomBERTModel()

for param in model.bert.parameters():
  param.requires_grad = False

In [48]:
training_args = TrainingArguments(
    num_train_epochs=20,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    output_dir=f'tmp/out',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir=f'tmp/logging',
    logging_steps=100,
    # save_steps=100,
    metric_for_best_model="eval_loss", 
    lr_scheduler_type='cosine_with_restarts',
    learning_rate=1e-2
)

model.train()
model.to('cuda:0')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

***** Running training *****
  Num examples = 209
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 540


Epoch,Training Loss,Validation Loss
1,1.575700,1.518717
2,1.500700,1.458981
3,1.397900,1.403892
4,1.335300,1.348324
5,1.185600,1.342650
6,1.132400,1.173709
7,1.131200,0.984324
8,1.006600,1.054731
9,0.905100,0.864719
10,0.940300,0.838577


***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 53
  Batch size = 8
***** Running Evaluation *****
  Num examples = 

TrainOutput(global_step=540, training_loss=0.9544233163197835, metrics={'train_runtime': 22.2159, 'train_samples_per_second': 188.154, 'train_steps_per_second': 24.307, 'total_flos': 0.0, 'train_loss': 0.9544233163197835, 'epoch': 20.0})

In [51]:
model.eval()

trainer = Trainer(
    model=model
)

y_pred = trainer.predict(val_dataset).predictions.argmax(axis=1).tolist()
print(accuracy_score(y_test, y_pred))

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 53
  Batch size = 8


0.9056603773584906
[2, 4, 4, 3, 4, 1, 2, 0, 3, 0, 4, 1, 0, 3, 0, 3, 3, 0, 2, 0, 0, 3, 4, 4, 0, 3, 1, 3, 1, 3, 3, 4, 4, 4, 3, 3, 4, 3, 2, 3, 1, 1, 2, 4, 3, 4, 4, 3, 4, 1, 3, 2, 3]
